In [82]:
import os
from openai import OpenAI
from dotenv import load_dotenv; load_dotenv(".env", override=True)
import numpy as np
import pandas as pd

In [83]:
# OpenAI models
EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-4o-mini"

### Creating Embeddings

In [84]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "<your api key if not set as env var>"))

# INPUT_TEXT = "I am a servant of this world."

# embedding_response = client.embeddings.create(input=INPUT_TEXT,model=EMBEDDING_MODEL)

# print(embedding_response.data[0].embedding)

### Chatting with LLMs

In [85]:
# USER_QUERY = "Which athletes won the most number of gold medals in 2024 Summer Olympics?"
# MESSAGES = [
#     {'role':'system', 'content': 'You answer questions provided by the user in a direct manner.'},
#     {'role':'user', 'content': USER_QUERY}
#             ]

# llm_response = client.chat.completions.create(
#     messages=MESSAGES,
#     model = GPT_MODEL,
#     temperature=0
# )

# print(llm_response.choices[0].message.content)

### Extracting Clean Text -> Chunking -> Embedding

In [86]:
# Register packages
import pymupdf.layout #important before importing pymupdf4llm
import pymupdf4llm
import pymupdf

# Open a PDF
doc = pymupdf.open(r"./doc-expert/documents/IS456-2000.pdf")
print(doc.page_count)

# Extract as Markdown
md = pymupdf4llm.to_markdown(doc, pages=None, embed_images=False, ignore_images=True, ignore_graphics=True, header=False, footer=False, show_progress=True)

107
Parsing 107 pages of './doc-expert/documents/IS456-2000.pdf'...


 32%|███▏      | 34/107 [00:36<00:47,  1.53it/s]

Performing OCR on page.number=34[35]...


100%|██████████| 107/107 [02:19<00:00,  1.30s/it]


Generating markdown text...


100%|██████████| 107/107 [00:00<00:00, 645.20it/s]


In [88]:
# Clean markdown
import re
import unicodedata

# 1. Remove replacement characters and normalize
md = unicodedata.normalize("NFKC", md)

# 2. Drop page headers/footers (common in standards, textbooks, codes)
md = re.sub(r"^IS\s*456\s*:\s*2000.*$", "", md, flags=re.MULTILINE)
md = re.sub(r"^\s*\d+\s*$", "", md, flags=re.MULTILINE)  # page numbers

# 3. Fix hyphenation across line breaks
md = re.sub(r"(\w+)-\n(\w+)", r"\1\2", md)

# 4. Merge broken lines that should be a single paragraph
md = re.sub(r"([^\n])\n(?!\n)", r"\1 ", md)

# 5. Collapse excessive blank lines
md = re.sub(r"\n{3,}", "\n\n", md)

# 6. Trim trailing whitespace
md_clean = re.sub(r"[ \t]+$", "", md, flags=re.MULTILINE)

# 7. Remove image removed comment 
md_clean = re.sub(r"==>\s*picture\s*\[\s*\d+\s*x\s*\d+\s*\]\s*intentionally\s*omitted\s*<==","",md_clean,flags=re.IGNORECASE)

# 7. Remove ****
md_clean = re.sub(r"^\*{2,}\s*$", "", md_clean, flags=re.MULTILINE)

# Save the output to an external file
from pathlib import Path
suffix = ".md"
Path(doc.name).with_suffix(suffix).write_bytes(md_clean.encode())

331142

### Chunking the markdown

In [94]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

def chunk_markdown(md, max_tokens=800):
    md = re.sub(r"(?=\*\*[A-Z])", r"\n", md)
    sections = re.split(r"(?:^|\n)(?=\*\*[A-Z][A-Za-z\s&]+\*\*)", md) #split before headings
    sections = [s for s in sections if s.strip()] # remove empty items
    chunks = []
    buf = []
    tot_token = 0
    
    for sec in sections:
        sec_token = len(enc.encode(sec))
        if tot_token + sec_token > max_tokens:
            chunks.append("".join(buf))
            buf, tot_token = [sec], sec_token
        else:
            buf.append(sec)
            tot_token += sec_token
            
    if buf:
        chunks.append("".join(buf))
    return chunks


chunks = chunk_markdown(md_clean)[1:]
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 124


In [95]:
print("len:", len(chunks))
for i, c in enumerate(chunks[:5]):
    print(i, type(c), repr(c)[:200])

len: 124
0 <class 'str'> '\n\n\n\n\n\n\n## _Indian Standard_\n\n## PLAIN AND REINFORCED CONCRETE - CODE OF PRACTICE ( _Fourth Revision )_\n\nICS 91.100.30\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n0 BIS 2000\n\nBUREAU OF INDIAN STANDARDS 
1 <class 'str'> '**CONTENTS**\n\n## PAGE\n\n|||||\n**SECTION 1 GENERAL**|\n**SECTION 1 GENERAL**|\n**SECTION 1 GENERAL**||| |---|---|---|---|---|---|---|---|---| |||||||||11| |1|SCOPE|||||||| |2|**REFERENCES**|||||||
2 <class 'str'> '**OF\n**DESIGNED<br>CONCRETE<br>Mrx||||29| |15.1|General|||29| |15.2|Frequency of Sampling|||29| |15.3|Test Specimen|||29| |15.4|Test Results of Sample|||29| |16 ACCEPTANCE<br>CRITERIA||||29| |17\n**
3 <class 'str'> '**AND\n**DETAILING|||||42| ||26.1|General||||42| ||26.2|Development<br>of Stress in Reinforcement||||42| ||26.3|Spacing of Reinforcement||||45| ||26.4|Nominal Cover to Reinforcement||||46| ||26.5|Req
4 <class 'str'> '**ANDDESIGN**<br>VALUES<br>AND PARTUL SAFEI”Y<br>**FACTORS**||67| ||**36.1**|**Characteristic Strength of\

### Create Embeddings

In [99]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "<your api key if not set as env var>"))

vectors = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=chunks).data
embeddings = [item.embedding for item in vectors]
print(len(embeddings))

124


### Store the embeddings in a vector database using **ChromaDB**

In [101]:
import chromadb
chroma = chromadb.Client()

collection = chroma.create_collection(
    name="my_code",
    metadata={"hnsw:space": "cosine"}
)

ids = [f"chunk-{i}" for i in range(len(chunks))]

collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=chunks,
    metadatas=[{"source":"IS4562000.pdf"} for _ in chunks]
)

In [105]:
USER_QUERY = "What is the maximum strain in concrete outermost compression fiber according to limit state of collapse in flexure?"
query_vec = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=USER_QUERY
).data[0].embedding

result = collection.query(
    query_embeddings=[query_vec],
    n_results=5
)


In [106]:
print(result['documents'][0][0])

**Serviceability**|| |||_DL_||_IL_|_WL_|_DL_|_IL_|_WL_| |_(1)_||_(2)_||_(3)_|_(4)_|_(5)_|_(6)_|_(7)_| |||_e _||_w _||||| |_DL+IL_|||_1.5_||
**I.0**|**1.0**|<br>**1.0**|| |_DL+WL_||**1.5 or**|||**1.5**|**1.0**||**1.0**| |||**$J”**||||||| |_DL+IL+_|_WL_|||_1.2_||**1.0**|_0.8_|_0.8_| |_NOTES_|||||||||

- **1 While considering earthquake effects, substitute EL for WL**

- **2 For the limit states of serviceability, the values of 7r given in this table m applicable for short tern effects. While assessing the long term effects due to creep the dead load and that pat of the live load likely to be permanent may only be consided.**


**I) This value is to be considered when stability against overturning or stress reversal is critical.**




   - b) The maximum strain in concrete at the outermost compression fibre is taken as 0.003 5 in bending.

   - cl The relationship between the compressive stress distribution in concrete and the strain in concrete may be assumed to be rectangle, trapezoid, 

In [107]:
RETRIEVED_DATA = result['documents'][0][0]

AUGMENTED_MSG = f"""
Based on this retrieved message from the knowledgebase, answer user's question.

Retrieved Message: {RETRIEVED_DATA}
User's question: {USER_QUERY}

Provide direct answers in 1-2 sentences.
"""

MESSAGES = [
    {'role':'system', 'content': 'You answer questions provided by the user in a direct manner.'},
    {'role':'user', 'content': AUGMENTED_MSG}]

llm_response = client.chat.completions.create(
    messages=MESSAGES,
    model = GPT_MODEL,
    temperature=0
)

print(llm_response.choices[0].message.content)

The maximum strain in concrete at the outermost compression fiber is taken as 0.0035 in bending according to the limit state of collapse in flexure.
